In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO
DATA_DIR='/home/users/sdunin/calc/TCST'

In [6]:
data = pd.read_csv('hamp_mutants_for_af2.csv')
msa = list(SeqIO.parse(f"{DATA_DIR}/offline_data//hamp/msa/final3.reb2.aln", "fasta"))


,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,L,T,I,T,Q,P,L,K,E,L,...,M,A,Q,R,L,Q,S,Y,E,E
1,L,I,I,T,R,P,I,R,E,L,...,M,A,E,Q,L,E,G,Y,A,A
2,L,T,I,T,R,P,I,K,E,L,...,M,A,E,R,L,A,S,Y,E,E
3,L,T,I,T,Q,P,I,K,E,L,...,M,A,E,R,L,A,S,Y,E,A
4,Q,T,I,T,K,P,I,R,E,L,...,M,A,S,Q,L,E,D,Y,D,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6451,W,P,V,F,R,D,L,N,H,L,...,V,S,H,Q,I,V,D,F,V,Q
6452,R,K,I,K,Q,E,L,Q,K,I,...,M,R,D,T,L,K,E,S,L,M
6453,W,L,I,I,R,P,I,R,Q,L,...,L,S,G,R,L,T,E,L,E,A
6454,R,N,V,S,R,P,V,K,L,L,...,L,S,N,R,L,N,Q,L,E,D


In [7]:
hampid2alnseq = pd.DataFrame(
        [(str(i.seq), i.id) for i in msa],
    columns=['sequence', 'id']
)

In [12]:
data

,idx,source_seq,mutants
0,0,LKELVQGVQRIIGELITSFNLM,LPELVQGVQRIIGELITSFNLM
1,0,LKELVQGVQRIIGELITSFNLM,LKELVQGWQRIIGELITSFNLM
2,0,LKELVQGVQRIIGELITSFNLM,LKELYQGVQRIIGELITSFNLM
3,0,LKELVQGVQRIIGELITSFNLM,LKELVQGFQRIIGELITSFNLM
4,0,LKELVQGVQRIIGELITSFNLM,LKELVQGVQWIIGELITSFNLM
...,...,...,...
86087,32,KMQVVRVVNAYKAQITETIDRV,KMQVVRVVNAYVAQITETIDRV
86088,32,KMQVVRVVNAYKAQITETIDRV,KMQVVRVVNAYKLQITETIDRV
86089,32,KMQVVRVVNAYKAQITETIDRV,KMQVVRVVNAYKDQITETIDRV
86090,32,KMQVVRVVNAYKAQITETIDRV,KMQVVRVVNAYKAQITEYIDRV


In [34]:
def find_seq(N,C, rest):
    for n, i in enumerate(rest):
        if i.find(N) > -1 and i.find(C) >-1:
            return n, i

In [38]:
def get_sequences(mutants, source):
    lm = 1
    mutants = mutants.groupby('idx').head(lm)
    to_fold = {'sequence':[], 'id': []}
    seqs = [x.replace('-', '') for x in source.sequence]
    for ind, row in mutants.iterrows():
        rw =  source.loc[row['idx']]['sequence']
        rw = rw.replace('-', '')
        seq = row['source_seq']
        mut = row['mutants']
        ww = len(seq)//2
        left, right = seq[:len(seq)//2], seq[len(seq)//2:]
        pos, rw = find_seq(left, right, seqs)
        N, C = rw.find(left), rw.find(right)
        if N > -1 and C > -1:
            new = rw[:N] + mut[:ww] + rw[N+ww:C] + mut[ww:] + rw[C+ww:]
            to_fold['sequence'].append(new + ':' + new)
            to_fold['id'].append(source.iloc[pos]['id'])
    return pd.DataFrame(to_fold)

In [39]:
to_fold = get_sequences(data, hampid2alnseq)

In [54]:
tpl = """#!/bin/bash
#SBATCH -p gpu
#SBATCH -n 8
#SBATCH --gres=gpu:1
#SBATCH --mem=24GB
#SBATCH -J {name}
#SBATCH -w edi08

source /opt/miniconda3/bin/activate cf_1.3

mkdir out/{name}
colabfold_batch in/{name}.csv out/{name} --model-type AlphaFold2-multimer-v2 --amber --zip
"""
def prepare_alpha_fold(data, batch_size=100, limit=1000, out='/home/nfs/awinski/af2_mutants/'):
    to_fold = data.sample(limit)
    names = []
    for n, i in enumerate(range(0, len(to_fold)+batch_size, batch_size)):
        inn = out + "in/"
        sh = out + '_SH/'
        part = to_fold[i:i+batch_size]
        name = f'hamp_{n}'
        names.append('sbatch _SH/' + name +'.sh')
        with open(sh + name + '.sh', 'w') as ss:
            ss.write(tpl.format(name=name))
        part.to_csv(inn + name + '.csv', index=False)
    run = '\n'.join(names)
    with open(out + 'batch.sh','w') as b:
        b.write('#!/bin/bash\n' + run +'\n')
        
    


In [55]:
prepare_alpha_fold(to_fold)